In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google/paligemma-3b-mix-224

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/paligemma-3b-mix-224)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="google/paligemma-3b-mix-224")

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForVision2Seq

processor = AutoProcessor.from_pretrained("google/paligemma-3b-mix-224")
model = AutoModelForVision2Seq.from_pretrained("google/paligemma-3b-mix-224")

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
"""
VIA CERVICAL CANCER CLASSIFIER - PALIGEMMA (PROPERLY FIXED)
Fixed the tensor dimension mismatch in the classifier.
This will work.
"""

# ============================================================================
# CELL 1: INSTALL
# ============================================================================
!pip install -q torch torchvision pandas numpy scikit-learn matplotlib seaborn pillow openpyxl tqdm transformers

print("✅ Packages installed")

# ============================================================================
# CELL 2: IMPORTS
# ============================================================================
import os, gc, re, json, random, warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, classification_report, confusion_matrix

warnings.filterwarnings('ignore')
print("✅ Imports complete")

# ============================================================================
# CELL 3: SETUP
# ============================================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

from google.colab import drive
drive.mount('/content/drive')

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA: {torch.cuda.is_available()}")
print("✅ Setup complete")

# ============================================================================
# CELL 4: CONFIG
# ============================================================================
class Config:
    VIA_IARC_ROOT = "/content/drive/MyDrive/JHPIEGO/IARCImageBankVIA"
    VIA_IARC_META = "/content/drive/MyDrive/JHPIEGO/IARCImageBankVIA/meta_data.xlsx"
    VIA_JHPIEGO_ROOT = "/content/drive/MyDrive/JHPIEGO/via_cropped+index"
    OUTPUT_DIR = "/content/drive/MyDrive/JHPIEGO/VIA_PaliGemma_Fixed"
    CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
    METRICS_DIR = os.path.join(OUTPUT_DIR, "metrics")

    MODEL_NAME = "google/paligemma-3b-mix-224"
    IMG_SIZE = 224
    BATCH_SIZE = 4
    EPOCHS = 10
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 0.01
    TEST_SIZE = 0.2
    VAL_SIZE = 0.1
    POSITIVE_CLASSES = ['Positive', 'Suspicious']
    DEVICE = torch.device("cuda")

cfg = Config()
os.makedirs(cfg.CHECKPOINT_DIR, exist_ok=True)
os.makedirs(cfg.METRICS_DIR, exist_ok=True)
print(f"✅ Config set")

# ============================================================================
# CELL 5: LOAD DATA
# ============================================================================
def load_via_datasets():
    JHPIEGO_LABELS = {
        'Negative': [3,4,6,9,12,14,15,16,19,22,26,27,28,30,31,33,34,37,40,41,44,47,48,50,51,53,54,56,57,58,60,62,66,67,71,74,75,76,79,83,84,86,88,94,95,97,103,104,105,106,108,109,111,114,115],
        'Positive': [1,2,8,10,11,13,17,18,20,21,23,24,29,32,35,36,38,39,42,43,45,46,49,52,59,61,63,64,68,69,70,72,73,77,78,80,81,82,85,87,90,91,92,93,96,98,100,101,102,107,110,112,113,116],
        'Suspicious': [5,7,25,55,65,89]
    }
    card_to_label = {card: label for label, cards in JHPIEGO_LABELS.items() for card in cards}

    jhpiego_records = []
    if os.path.exists(cfg.VIA_JHPIEGO_ROOT):
        for filename in os.listdir(cfg.VIA_JHPIEGO_ROOT):
            if not filename.startswith("via_img_"): continue
            match = re.search(r'via_img_(\d+)', filename)
            if match:
                card_num = int(match.group(1))
                if card_num in card_to_label:
                    jhpiego_records.append({
                        "CaseNumber": f"JHPIEGO_{card_num}",
                        "image_path": os.path.join(cfg.VIA_JHPIEGO_ROOT, filename),
                        "VIA_simple": card_to_label[card_num],
                        "Source": "JHPIEGO"
                    })
        jhpiego_data = pd.DataFrame(jhpiego_records)
        print(f"JHPIEGO: {len(jhpiego_data)}")
    else:
        jhpiego_data = pd.DataFrame()

    iarc_data = pd.DataFrame()
    if os.path.exists(cfg.VIA_IARC_META):
        iarc_meta = pd.read_excel(cfg.VIA_IARC_META)
        iarc_meta["CaseNumber"] = iarc_meta["CaseNumber"].astype(int)

        def simplify_via(x):
            s = str(x).strip().lower()
            if "suspicious" in s: return "Suspicious"
            elif "positive" in s: return "Positive"
            else: return "Negative"

        iarc_meta["VIA_simple"] = iarc_meta["VIA"].apply(simplify_via)

        iarc_records = []
        for entry in os.listdir(cfg.VIA_IARC_ROOT):
            case_dir = os.path.join(cfg.VIA_IARC_ROOT, entry)
            if not os.path.isdir(case_dir): continue
            digits = re.findall(r"\d+", entry)
            if not digits: continue
            case_num = int(digits[0])

            for f in os.listdir(case_dir):
                if f.lower().endswith(('.jpg', '.jpeg', '.png')):
                    if f.replace('.jpg', '').replace('.jpeg', '').replace('.png', '').endswith("1"):
                        iarc_records.append({
                            "CaseNumber": case_num,
                            "image_path": os.path.join(case_dir, f),
                            "Source": "IARC"
                        })
                        break

        iarc_images = pd.DataFrame(iarc_records)
        iarc_data = pd.merge(iarc_meta, iarc_images, on="CaseNumber", how="inner")
        iarc_data["CaseNumber"] = iarc_data["CaseNumber"].astype(str)
        print(f"IARC: {len(iarc_data)}")

    combined = pd.concat([iarc_data, jhpiego_data], ignore_index=True)
    combined['binary_label'] = combined['VIA_simple'].apply(lambda x: 'Abnormal' if x in cfg.POSITIVE_CLASSES else 'Normal')
    combined['label'] = (combined['binary_label'] == 'Abnormal').astype(int)
    valid_mask = combined['image_path'].apply(os.path.isfile)
    combined = combined[valid_mask].reset_index(drop=True)
    return combined

data_df = load_via_datasets()
print(f"\nTotal: {len(data_df)} | Normal: {(data_df['label']==0).sum()} | Abnormal: {(data_df['label']==1).sum()}")

train_val_df, test_df = train_test_split(data_df, test_size=cfg.TEST_SIZE, random_state=SEED, stratify=data_df['label'])
train_df, val_df = train_test_split(train_val_df, test_size=cfg.VAL_SIZE, random_state=SEED, stratify=train_val_df['label'])
print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
print("✅ Data loaded")

# ============================================================================
# CELL 6: DATASET
# ============================================================================
class VIADataset(Dataset):
    def __init__(self, dataframe, processor, img_size=224):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.img_size = img_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            image = Image.open(row['image_path']).convert('RGB')
            if image.size != (self.img_size, self.img_size):
                image = image.resize((self.img_size, self.img_size), Image.LANCZOS)
        except:
            image = Image.new('RGB', (self.img_size, self.img_size), (128, 128, 128))

        inputs = self.processor(images=image, return_tensors="pt")
        return {
            'pixel_values': inputs['pixel_values'].squeeze(0),
            'label': torch.tensor(int(row['label']), dtype=torch.long)
        }

print("✅ Dataset class created")

# ============================================================================
# CELL 7: LOAD PALIGEMMA
# ============================================================================
torch.cuda.empty_cache()
gc.collect()

print("Loading PaliGemma...")
processor = AutoProcessor.from_pretrained(cfg.MODEL_NAME)
print("  ✓ Processor loaded")

paligemma_model = PaliGemmaForConditionalGeneration.from_pretrained(
    cfg.MODEL_NAME,
    torch_dtype=torch.float16,
    # Removed device_map="auto"
).to(cfg.DEVICE) # Explicitly move the model to the device
print("  ✓ Model loaded")
print(f"GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print("✅ PaliGemma ready")

# ============================================================================
# CELL 8: CLASSIFIER (FIXED)
# ============================================================================
class PaliGemmaClassifier(nn.Module):
    """Fixed classifier with proper tensor handling"""

    def __init__(self, base_model, num_classes=2):
        super().__init__()
        self.vision_model = base_model.vision_tower

        # Freeze vision encoder
        for param in self.vision_model.parameters():
            param.requires_grad = False

        print("Detecting architecture...")

        # Test forward pass to understand output structure
        with torch.no_grad():
            # Ensure dummy input is float32
            dummy = torch.randn(1, 3, 224, 224, dtype=torch.float32).to(cfg.DEVICE)
            out = self.vision_model(dummy)

            # Extract features
            if hasattr(out, 'last_hidden_state'):
                features = out.last_hidden_state
            elif hasattr(out, 'pooler_output'):
                features = out.pooler_output
            elif isinstance(out, tuple):
                features = out[0]
            else:
                features = out

            # Ensure features are float32
            features = features.float()

            print(f"  Vision output shape: {features.shape}")
            print(f"  Vision output dtype: {features.dtype}")

            # Determine dimensions
            if len(features.shape) == 3:
                # [batch, sequence, features]
                batch_size, seq_len, feature_dim = features.shape
                self.feature_dim = feature_dim
                self.use_pooling = True
                print(f"  Has sequence: True (seq_len={seq_len})")
            elif len(features.shape) == 2:
                # [batch, features]
                batch_size, feature_dim = features.shape
                self.feature_dim = feature_dim
                self.use_pooling = False
                print(f"  Has sequence: False")
            else:
                raise ValueError(f"Unexpected feature shape: {features.shape}")

        print(f"  Feature dim: {self.feature_dim}")

        # Build classifier
        if self.use_pooling:
            # Need to pool sequence dimension
            self.pool = nn.AdaptiveAvgPool1d(1)
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(self.feature_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(256, num_classes)
            )
        else:
            # Already pooled
            self.classifier = nn.Sequential(
                nn.Linear(self.feature_dim, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(256, num_classes)
            )

    def forward(self, pixel_values):
        """Forward pass with proper error handling"""

        # Ensure input is float32 and on the correct device
        # NOTE: Removed the .to(self.vision_model.dtype) part as the model weights are now on GPU float16
        pixel_values = pixel_values.to(pixel_values.device)

        # Extract features (frozen)
        with torch.no_grad():
            out = self.vision_model(pixel_values)

            if hasattr(out, 'last_hidden_state'):
                features = out.last_hidden_state
            elif hasattr(out, 'pooler_output'):
                features = out.pooler_output
            elif isinstance(out, tuple):
                features = out[0]
            else:
                features = out

        # Ensure features are in float32 for classifier
        features = features.float()

        # Pool if needed
        if self.use_pooling:
            # Shape: [batch, seq, features] -> [batch, features, seq]
            features = features.transpose(1, 2)
            # Pool: [batch, features, seq] -> [batch, features, 1]
            features = self.pool(features)
            # Flatten will handle the rest

        # Classify
        logits = self.classifier(features)

        return logits

# Create classifier
print("\nCreating classifier...")
classifier = PaliGemmaClassifier(paligemma_model, num_classes=2).to(cfg.DEVICE)

trainable = sum(p.numel() for p in classifier.parameters() if p.requires_grad)
total = sum(p.numel() for p in classifier.parameters())
print(f"\nTrainable: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"Total: {total:,}")
print("✅ Classifier ready")

# ============================================================================
# CELL 9: DATA LOADERS
# ============================================================================
train_dataset = VIADataset(train_df, processor, cfg.IMG_SIZE)
val_dataset = VIADataset(val_df, processor, cfg.IMG_SIZE)
test_dataset = VIADataset(test_df, processor, cfg.IMG_SIZE)

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([item['pixel_values'] for item in batch]),
        'labels': torch.stack([item['label'] for item in batch])
    }

train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=cfg.BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True)

print(f"Train: {len(train_loader)} batches | Val: {len(val_loader)} batches | Test: {len(test_loader)} batches")
print("✅ Data loaders ready")

# ============================================================================
# CELL 10: TRAINING SETUP
# ============================================================================
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW([p for p in classifier.parameters() if p.requires_grad], lr=cfg.LEARNING_RATE, weight_decay=cfg.WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.EPOCHS, eta_min=1e-6)
print("✅ Training setup complete")

# ============================================================================
# CELL 11: TRAINING LOOP
# ============================================================================
print("="*80)
print("STARTING TRAINING")
print("="*80)

best_val_f1 = 0
patience_counter = 0

for epoch in range(cfg.EPOCHS):
    print(f"\n{'='*60}\nEpoch {epoch+1}/{cfg.EPOCHS}\n{'='*60}")

    # TRAIN
    classifier.train()
    train_loss = 0
    train_preds, train_labels = [], []

    pbar = tqdm(train_loader, desc="Training")
    for batch in pbar:
        try:
            pixel_values = batch['pixel_values'].to(cfg.DEVICE)
            labels = batch['labels'].to(cfg.DEVICE)

            optimizer.zero_grad()
            logits = classifier(pixel_values)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            train_preds.extend(preds)
            train_labels.extend(labels.cpu().numpy())

            pbar.set_postfix({'loss': f'{train_loss/(pbar.n+1):.4f}'})
        except Exception as e:
            print(f"\nError in training batch: {e}")
            continue

    train_acc = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='binary')
    print(f"Train - Loss: {train_loss/len(train_loader):.4f} | Acc: {train_acc*100:.2f}% | F1: {train_f1:.4f}")

    # VALIDATE
    classifier.eval()
    val_loss = 0
    val_preds, val_labels = [], []

    with torch.no_grad():
        pbar = tqdm(val_loader, desc="Validation")
        for batch in pbar:
            try:
                pixel_values = batch['pixel_values'].to(cfg.DEVICE)
                labels = batch['labels'].to(cfg.DEVICE)

                logits = classifier(pixel_values)
                loss = criterion(logits, labels)

                val_loss += loss.item()
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())

                pbar.set_postfix({'loss': f'{val_loss/(pbar.n+1):.4f}'})
            except Exception as e:
                print(f"\nError in validation batch: {e}")
                continue

    val_acc = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='binary')
    print(f"Val   - Loss: {val_loss/len(val_loader):.4f} | Acc: {val_acc*100:.2f}% | F1: {val_f1:.4f}")

    # Save best
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': classifier.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc
        }, os.path.join(cfg.CHECKPOINT_DIR, 'best_model.pth'))
        print(f"✅ Saved (F1: {val_f1:.4f})")
    else:
        patience_counter += 1
        if patience_counter >= 5:
            print("⏹️ Early stopping")
            break

    scheduler.step()
    torch.cuda.empty_cache()

print(f"\n{'='*80}\nTRAINING COMPLETED\n{'='*80}")
print(f"Best Val F1: {best_val_f1:.4f}")

# ============================================================================
# CELL 12: TEST EVALUATION
# ============================================================================
print(f"\n{'='*80}\nTEST EVALUATION\n{'='*80}")

checkpoint = torch.load(os.path.join(cfg.CHECKPOINT_DIR, 'best_model.pth'))
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier.eval()

test_preds, test_labels, test_probs = [], [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        pixel_values = batch['pixel_values'].to(cfg.DEVICE)
        labels = batch['labels'].to(cfg.DEVICE)

        logits = classifier(pixel_values)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='binary')
test_prec = precision_score(test_labels, test_preds, average='binary')
test_rec = recall_score(test_labels, test_preds, average='binary')
test_auc = roc_auc_score(test_labels, test_probs)

print(f"\n{'='*60}\nTEST RESULTS\n{'='*60}")
print(f"Accuracy:  {test_acc*100:.2f}%")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print(f"AUC-ROC:   {test_auc:.4f}")
print(f"\nConfusion Matrix:\n{confusion_matrix(test_labels, test_preds)}")
print(f"\n{classification_report(test_labels, test_preds, target_names=['Normal', 'Abnormal'])}")
print("="*60)

results = {'test_accuracy': float(test_acc), 'test_f1': float(test_f1), 'test_precision': float(test_prec), 'test_recall': float(test_rec), 'test_auc': float(test_auc)}
with open(os.path.join(cfg.METRICS_DIR, 'results.json'), 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n✅ Results saved to {cfg.METRICS_DIR}")

# ============================================================================
# CELL 13: SAMPLE PREDICTIONS
# ============================================================================
print(f"\n{'='*80}\nSAMPLE PREDICTIONS\n{'='*80}")

indices = np.random.choice(len(test_df), min(5, len(test_df)), replace=False)

classifier.eval()
with torch.no_grad():
    for i, idx in enumerate(indices):
        row = test_df.iloc[idx]
        image = Image.open(row['image_path']).convert('RGB')
        inputs = processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].to(cfg.DEVICE)

        logits = classifier(pixel_values)
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(logits, dim=1).item()
        conf = probs[0, pred].item()

        true_label = "Normal" if row['label'] == 0 else "Abnormal"
        pred_label = "Normal" if pred == 0 else "Abnormal"
        status = "✅" if pred == row['label'] else "❌"

        print(f"{status} Sample {i+1}: True={true_label}, Pred={pred_label} ({conf*100:.1f}%)")

print(f"\n{'='*80}\nALL COMPLETE!\n{'='*80}")
print(f"Final Performance:")
print(f"  Accuracy:  {test_acc*100:.2f}%")
print(f"  F1 Score:  {test_f1:.4f}")
print(f"  AUC-ROC:   {test_auc:.4f}")
print(f"\nSaved to: {cfg.OUTPUT_DIR}")
print("="*80)

✅ Packages installed
✅ Imports complete
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: Tesla T4
CUDA: True
✅ Setup complete
✅ Config set
JHPIEGO: 121
IARC: 185

Total: 306 | Normal: 150 | Abnormal: 156
Train: 219 | Val: 25 | Test: 62
✅ Data loaded
✅ Dataset class created
Loading PaliGemma...
  ✓ Processor loaded


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

  ✓ Model loaded
GPU Memory: 14.44 GB
✅ PaliGemma ready

Creating classifier...
Detecting architecture...
  Vision output shape: torch.Size([1, 256, 1152])
  Vision output dtype: torch.float32
  Has sequence: True (seq_len=256)
  Feature dim: 1152

Trainable: 722,178 (0.17%)
Total: 413,164,530
✅ Classifier ready
Train: 55 batches | Val: 7 batches | Test: 16 batches
✅ Data loaders ready
✅ Training setup complete
STARTING TRAINING

Epoch 1/10


Training:   0%|          | 0/55 [00:00<?, ?it/s]

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

...

Testing:   0%|          | 0/16 [00:00<?, ?it/s]

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y


TEST RESULTS
Accuracy:  80.65%
Precision: 0.7632
Recall:    0.9062
F1 Score:  0.8286
AUC-ROC:   0.8479

Confusion Matrix:
[[21  9]
 [ 3 29]]

              precision    recall  f1-score   support

      Normal       0.88      0.70      0.78        30
    Abnormal       0.76      0.91      0.83        32

    accuracy                           0.81        62
   macro avg       0.82      0.80      0.80        62
weighted avg       0.82      0.81      0.80        62


✅ Results saved to /content/drive/MyDrive/VIA_PaliGemma_Fixed/metrics

SAMPLE PREDICTIONS
✅ Sample 1: True=Normal, Pred=Normal (88.0%)
✅ Sample 2: True=Abnormal, Pred=Abnormal (68.8%)
❌ Sample 3: True=Normal, Pred=Abnormal (87.6%)
✅ Sample 4: True=Abnormal, Pred=Abnormal (92.3%)


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


✅ Sample 5: True=Abnormal, Pred=Abnormal (76.9%)

ALL COMPLETE!
Final Performance:
  Accuracy:  80.65%
  F1 Score:  0.8286
  AUC-ROC:   0.8479

Saved to: /content/drive/MyDrive/VIA_PaliGemma_Fixed


V2

In [ ]:
"""
=============================================================================
IMPROVED VIA CERVICAL CANCER CLASSIFIER - ENHANCED VERSION
=============================================================================
This implementation includes:
1. Data augmentation for better generalization
2. Improved architecture with BatchNorm and better dropout
3. Optimized learning rate schedule with OneCycleLR
4. Test-Time Augmentation (TTA) for robust predictions
5. Better model checkpointing and evaluation
=============================================================================
"""

# ===========================================================================
# CELL 1: ENHANCED CONFIGURATION
# ===========================================================================
class ImprovedConfig:
    """Enhanced configuration with optimized hyperparameters"""
    # Data paths (same as before)
    VIA_IARC_ROOT = "/content/drive/MyDrive/JHPIEGO/IARCImageBankVIA"
    VIA_IARC_META = "/content/drive/MyDrive/JHPIEGO/IARCImageBankVIA/meta_data.xlsx"
    VIA_JHPIEGO_ROOT = "/content/drive/MyDrive/JHPIEGO/via_cropped+index"
    OUTPUT_DIR = "/content/drive/MyDrive/JHPIEGO/VIA_PaliGemma_Improved"
    CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
    METRICS_DIR = os.path.join(OUTPUT_DIR, "metrics")

    # Model settings
    MODEL_NAME = "google/paligemma-3b-mix-224"
    IMG_SIZE = 224

    # Training hyperparameters (OPTIMIZED)
    BATCH_SIZE = 4
    EPOCHS = 15  # Increased from 10
    LEARNING_RATE = 3e-4  # Lowered from 1e-3
    MAX_LR = 5e-4  # For OneCycleLR
    WEIGHT_DECAY = 0.01

    # Data splits
    TEST_SIZE = 0.2
    VAL_SIZE = 0.1

    # Class definitions
    POSITIVE_CLASSES = ['Positive', 'Suspicious']

    # Device
    DEVICE = torch.device("cuda")

    # Random seed
    SEED = 42

# Create configuration instance
imp_cfg = ImprovedConfig()
os.makedirs(imp_cfg.CHECKPOINT_DIR, exist_ok=True)
os.makedirs(imp_cfg.METRICS_DIR, exist_ok=True)
print("✅ Enhanced configuration created")

# ===========================================================================
# CELL 2: IMPROVED DATASET WITH AUGMENTATION
# ===========================================================================
# Import the necessary module for transforms
from torchvision import transforms

class ImprovedVIADataset(Dataset):
    """
    Enhanced dataset with medical-appropriate data augmentation.

    Augmentations include:
    - Geometric: flips, rotations, affine transforms (valid for cervical images)
    - Color: brightness, contrast, saturation (important for VIA color changes)
    - Auto-contrast for lighting normalization
    """
    def __init__(self, dataframe, processor, img_size=224, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.img_size = img_size
        self.augment = augment

        # Define augmentation pipeline for training
        if augment:
            self.transform = transforms.Compose([
                # Geometric augmentations - all valid for cervical images
                # Images can be viewed from any orientation
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=20),

                # Random affine for slight position/scale variations
                transforms.RandomAffine(
                    degrees=0,  # Rotation handled above
                    translate=(0.1, 0.1),  # 10% shift
                    scale=(0.9, 1.1),  # 10% zoom
                    shear=5  # Slight shearing
                ),

                # Color augmentations - CRITICAL for VIA
                # VIA diagnosis depends on acetowhite color changes
                transforms.ColorJitter(
                    brightness=0.3,  # Lighting variations between exams
                    contrast=0.3,    # Camera/exposure differences
                    saturation=0.2,  # Color intensity variations
                    hue=0.05         # Slight hue shifts (limited to preserve diagnostic info)
                ),

                # Automatic contrast adjustment - normalizes lighting
                transforms.RandomAutocontrast(p=0.3),
            ])
        else:
            self.transform = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load and resize image
        try:
            image = Image.open(row['image_path']).convert('RGB')
            if image.size != (self.img_size, self.img_size):
                image = image.resize((self.img_size, self.img_size), Image.LANCZOS)
        except Exception as e:
            # Fallback: create gray image if loading fails
            print(f"Warning: Failed to load {row['image_path']}: {e}")
            image = Image.new('RGB', (self.img_size, self.img_size), (128, 128, 128))

        # Apply augmentation BEFORE processor
        # This is crucial - augment PIL image, then process for model
        if self.transform:
            image = self.transform(image)

        # Process image for PaliGemma
        inputs = self.processor(images=image, return_tensors="pt")

        return {
            'pixel_values': inputs['pixel_values'].squeeze(0),
            'label': torch.tensor(int(row['label']), dtype=torch.long)
        }

print("✅ Improved dataset class created")

# ===========================================================================
# CELL 3: ENHANCED CLASSIFIER ARCHITECTURE
# ===========================================================================
class ImprovedPaliGemmaClassifier(nn.Module):
    """
    Enhanced classifier with:
    - Deeper architecture (4 hidden layers vs 3)
    - Batch normalization for stable training
    - Graduated dropout (higher in early layers, lower in later layers)
    - Better capacity for learning subtle features
    """
    def __init__(self, base_model, num_classes=2):
        super().__init__()

        # Use frozen PaliGemma vision encoder
        self.vision_model = base_model.vision_tower

        # Freeze all vision model parameters
        # We only train the classifier head
        for param in self.vision_model.parameters():
            param.requires_grad = False

        print("Detecting vision model architecture...")

        # Test forward pass to determine output dimensions
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224, dtype=torch.float32).to(imp_cfg.DEVICE)
            out = self.vision_model(dummy)

            # Extract features based on output type
            if hasattr(out, 'last_hidden_state'):
                features = out.last_hidden_state
            elif hasattr(out, 'pooler_output'):
                features = out.pooler_output
            elif isinstance(out, tuple):
                features = out[0]
            else:
                features = out

            # Ensure float32 for classifier
            features = features.float()

            print(f"  Vision output shape: {features.shape}")
            print(f"  Vision output dtype: {features.dtype}")

            # Determine if we need sequence pooling
            if len(features.shape) == 3:
                # [batch, sequence, features]
                batch_size, seq_len, feature_dim = features.shape
                self.feature_dim = feature_dim
                self.use_pooling = True
                print(f"  Sequence length: {seq_len}")
            elif len(features.shape) == 2:
                # [batch, features]
                batch_size, feature_dim = features.shape
                self.feature_dim = feature_dim
                self.use_pooling = False
            else:
                raise ValueError(f"Unexpected feature shape: {features.shape}")

        print(f"  Feature dimension: {self.feature_dim}")

        # Build improved classifier head
        if self.use_pooling:
            # Pool sequence dimension if needed
            self.pool = nn.AdaptiveAvgPool1d(1)

            # 4-layer classifier with BatchNorm and graduated dropout
            # Dropout: 0.4 -> 0.35 -> 0.3 -> 0.2 (decreasing)
            self.classifier = nn.Sequential(
                nn.Flatten(),

                # Layer 1: feature_dim -> 768
                nn.Linear(self.feature_dim, 768),
                nn.BatchNorm1d(768),  # Stabilizes training
                nn.ReLU(inplace=True),
                nn.Dropout(0.4),  # Highest dropout in first layer

                # Layer 2: 768 -> 512
                nn.Linear(768, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.35),

                # Layer 3: 512 -> 256
                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),

                # Layer 4: 256 -> 128
                nn.Linear(256, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),  # Lower dropout before output

                # Output layer
                nn.Linear(128, num_classes)
            )
        else:
            # Same architecture without pooling
            self.classifier = nn.Sequential(
                nn.Linear(self.feature_dim, 768),
                nn.BatchNorm1d(768),
                nn.ReLU(inplace=True),
                nn.Dropout(0.4),

                nn.Linear(768, 512),
                nn.BatchNorm1d(512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.35),

                nn.Linear(512, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),

                nn.Linear(256, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),

                nn.Linear(128, num_classes)
            )

    def forward(self, pixel_values):
        """
        Forward pass with proper dtype handling

        Args:
            pixel_values: Input images [batch, channels, height, width]

        Returns:
            logits: Class predictions [batch, num_classes]
        """
        # Ensure correct device
        pixel_values = pixel_values.to(pixel_values.device)

        # Extract frozen features
        with torch.no_grad():
            out = self.vision_model(pixel_values)

            # Get features based on output structure
            if hasattr(out, 'last_hidden_state'):
                features = out.last_hidden_state
            elif hasattr(out, 'pooler_output'):
                features = out.pooler_output
            elif isinstance(out, tuple):
                features = out[0]
            else:
                features = out

        # Convert to float32 for classifier
        features = features.float()

        # Pool sequence dimension if needed
        if self.use_pooling:
            # [batch, seq, features] -> [batch, features, seq]
            features = features.transpose(1, 2)
            # [batch, features, seq] -> [batch, features, 1]
            features = self.pool(features)
            # Flatten handles the rest

        # Pass through classifier
        logits = self.classifier(features)

        return logits

print("✅ Improved classifier architecture defined")

# ===========================================================================
# CELL 4: CREATE IMPROVED DATASETS AND LOADERS
# ===========================================================================
print("\n" + "="*80)
print("CREATING IMPROVED DATASETS")
print("="*80)

# Reuse the same data splits from before
# We already have train_df, val_df, test_df from the original code

# Create datasets with augmentation
print("Creating datasets with augmentation...")
imp_train_dataset = ImprovedVIADataset(
    train_df,
    processor,
    imp_cfg.IMG_SIZE,
    augment=True  # Augmentation ON for training
)
imp_val_dataset = ImprovedVIADataset(
    val_df,
    processor,
    imp_cfg.IMG_SIZE,
    augment=False  # Augmentation OFF for validation
)
imp_test_dataset = ImprovedVIADataset(
    test_df,
    processor,
    imp_cfg.IMG_SIZE,
    augment=False  # Augmentation OFF for testing
)

print(f"✅ Train dataset: {len(imp_train_dataset)} samples (with augmentation)")
print(f"✅ Val dataset: {len(imp_val_dataset)} samples (no augmentation)")
print(f"✅ Test dataset: {len(imp_test_dataset)} samples (no augmentation)")

# Collate function (same as before)
def collate_fn(batch):
    """Collate function to batch samples"""
    return {
        'pixel_values': torch.stack([item['pixel_values'] for item in batch]),
        'labels': torch.stack([item['label'] for item in batch])
    }

# Create data loaders
imp_train_loader = DataLoader(
    imp_train_dataset,
    batch_size=imp_cfg.BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)

imp_val_loader = DataLoader(
    imp_val_dataset,
    batch_size=imp_cfg.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)

imp_test_loader = DataLoader(
    imp_test_dataset,
    batch_size=imp_cfg.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)

print(f"✅ Train loader: {len(imp_train_loader)} batches")
print(f"✅ Val loader: {len(imp_val_loader)} batches")
print(f"✅ Test loader: {len(imp_test_loader)} batches")

# ===========================================================================
# CELL 5: CREATE IMPROVED MODEL
# ===========================================================================
print("\n" + "="*80)
print("CREATING IMPROVED MODEL")
print("="*80)

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Create improved classifier
print("Building improved classifier...")
imp_classifier = ImprovedPaliGemmaClassifier(
    paligemma_model,  # Reuse the loaded PaliGemma model
    num_classes=2
).to(imp_cfg.DEVICE)

# Print parameter counts
trainable = sum(p.numel() for p in imp_classifier.parameters() if p.requires_grad)
total = sum(p.numel() for p in imp_classifier.parameters())
print(f"\n📊 Model Statistics:")
print(f"  Trainable parameters: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"  Total parameters: {total:,}")
print(f"  GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print("✅ Improved classifier ready")

# ===========================================================================
# CELL 6: SETUP TRAINING WITH IMPROVED OPTIMIZER & SCHEDULER
# ===========================================================================
print("\n" + "="*80)
print("SETTING UP IMPROVED TRAINING")
print("="*80)

# Loss function (standard CrossEntropyLoss - no class weighting needed)
criterion = nn.CrossEntropyLoss()

# Improved optimizer - AdamW with better hyperparameters
imp_optimizer = torch.optim.AdamW(
    [p for p in imp_classifier.parameters() if p.requires_grad],
    lr=imp_cfg.LEARNING_RATE,  # 3e-4 (lower than before)
    weight_decay=imp_cfg.WEIGHT_DECAY,
    betas=(0.9, 0.999),
    eps=1e-8
)

# OneCycleLR scheduler - better than CosineAnnealingLR
# Provides warmup, peak, and gradual decay
imp_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    imp_optimizer,
    max_lr=imp_cfg.MAX_LR,  # Peak at 5e-4
    epochs=imp_cfg.EPOCHS,
    steps_per_epoch=len(imp_train_loader),
    pct_start=0.3,  # 30% of training for warmup
    anneal_strategy='cos',
    div_factor=25.0,  # initial_lr = max_lr/25 = 2e-5
    final_div_factor=10000.0  # min_lr = initial_lr/10000
)

print(f"✅ Optimizer: AdamW")
print(f"   Initial LR: {imp_cfg.LEARNING_RATE:.2e}")
print(f"   Max LR: {imp_cfg.MAX_LR:.2e}")
print(f"   Weight decay: {imp_cfg.WEIGHT_DECAY}")
print(f"✅ Scheduler: OneCycleLR")
print(f"   Warmup: {int(0.3 * imp_cfg.EPOCHS)} epochs")
print(f"   Steps per epoch: {len(imp_train_loader)}")

# ===========================================================================
# CELL 7: IMPROVED TRAINING LOOP
# ===========================================================================
print("\n" + "="*80)
print("STARTING IMPROVED TRAINING")
print("="*80)

# Training tracking
best_val_f1 = 0.0
patience_counter = 0
patience_limit = 5
train_history = []
val_history = []

# Define checkpoint path for the best model
best_checkpoint_path = os.path.join(imp_cfg.CHECKPOINT_DIR, 'best_improved_model.pth')
last_checkpoint_path = os.path.join(imp_cfg.CHECKPOINT_DIR, 'last_improved_model.pth')


for epoch in range(imp_cfg.EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{imp_cfg.EPOCHS}")
    print(f"{'='*60}")

    # ===== TRAINING PHASE =====
    imp_classifier.train()
    train_loss = 0.0
    train_preds, train_labels = [], []

    pbar = tqdm(imp_train_loader, desc="Training")
    for batch_idx, batch in enumerate(pbar):
        # Move data to device
        pixel_values = batch['pixel_values'].to(imp_cfg.DEVICE)
        labels = batch['labels'].to(imp_cfg.DEVICE)

        # Forward pass
        imp_optimizer.zero_grad()
        logits = imp_classifier(pixel_values)
        loss = criterion(logits, labels)

        # Backward pass
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(imp_classifier.parameters(), max_norm=1.0)

        # Optimizer step
        imp_optimizer.step()

        # Scheduler step (per batch, not per epoch)
        imp_scheduler.step()

        # Track metrics
        train_loss += loss.item()
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        train_preds.extend(preds)
        train_labels.extend(labels.cpu().numpy())

        # Update progress bar
        current_lr = imp_optimizer.param_groups[0]['lr']
        pbar.set_postfix({
            'loss': f'{train_loss/(batch_idx+1):.4f}',
            'lr': f'{current_lr:.2e}'
        })

    # Calculate training metrics
    train_acc = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='binary')
    train_prec = precision_score(train_labels, train_preds, average='binary')
    train_rec = recall_score(train_labels, train_preds, average='binary')

    print(f"Train - Loss: {train_loss/len(imp_train_loader):.4f} | "
          f"Acc: {train_acc*100:.2f}% | F1: {train_f1:.4f} | "
          f"Prec: {train_prec:.4f} | Rec: {train_rec:.4f}")

    # ===== VALIDATION PHASE =====
    imp_classifier.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        pbar = tqdm(imp_val_loader, desc="Validation")
        for batch_idx, batch in enumerate(pbar):
            pixel_values = batch['pixel_values'].to(imp_cfg.DEVICE)
            labels = batch['labels'].to(imp_cfg.DEVICE)

            # Forward pass only
            logits = imp_classifier(pixel_values)
            loss = criterion(logits, labels)

            # Track metrics
            val_loss += loss.item()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())

            pbar.set_postfix({'loss': f'{val_loss/(batch_idx+1):.4f}'})

    # Calculate validation metrics
    val_acc = accuracy_score(val_labels, val_preds)
    val_f1 = f1_score(val_labels, val_preds, average='binary')
    val_prec = precision_score(val_labels, val_preds, average='binary')
    val_rec = recall_score(val_labels, val_preds, average='binary')

    print(f"Val   - Loss: {val_loss/len(imp_val_loader):.4f} | "
          f"Acc: {val_acc*100:.2f}% | F1: {val_f1:.4f} | "
          f"Prec: {val_prec:.4f} | Rec: {val_rec:.4f}")

    # Save training history
    train_history.append({
        'epoch': epoch + 1,
        'loss': train_loss/len(imp_train_loader),
        'acc': train_acc,
        'f1': train_f1,
        'prec': train_prec,
        'rec': train_rec
    })

    val_history.append({
        'epoch': epoch + 1,
        'loss': val_loss/len(imp_val_loader),
        'acc': val_acc,
        'f1': val_f1,
        'prec': val_prec,
        'rec': val_rec
    })

    # ===== MODEL CHECKPOINTING =====
    # Save best model based on validation F1 score
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0

        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': imp_classifier.state_dict(),
            'optimizer_state_dict': imp_optimizer.state_dict(),
            'scheduler_state_dict': imp_scheduler.state_dict(),
            'val_f1': val_f1,
            'val_acc': val_acc,
            'val_prec': val_prec,
            'val_rec': val_rec
        }, best_checkpoint_path)

        print(f"✅ Saved best model (F1: {val_f1:.4f})")
    else:
        patience_counter += 1
        print(f"⏸️  No improvement ({patience_counter}/{patience_limit})")

        if patience_counter >= patience_limit:
            print(f"⏹️  Early stopping triggered")
            break

    # Save the last model at the end of each epoch
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': imp_classifier.state_dict(),
        'optimizer_state_dict': imp_optimizer.state_dict(),
        'scheduler_state_dict': imp_scheduler.state_dict(),
        'val_f1': val_f1,
        'val_acc': val_acc,
        'val_prec': val_prec,
        'val_rec': val_rec
    }, last_checkpoint_path)
    print(f"✅ Saved last model")

    # Clear GPU memory
    torch.cuda.empty_cache()

print(f"\n{'='*80}")
print("TRAINING COMPLETED")
print(f"{'='*80}")
print(f"Best Validation F1: {best_val_f1:.4f}")

# ===========================================================================
# CELL 8: TEST EVALUATION (STANDARD)
# ===========================================================================
print(f"\n{'='*80}")
print("TEST EVALUATION (STANDARD)")
print(f"{'='*80}")

# Load best model (or last model if no improvement was saved)
checkpoint_path_to_load = best_checkpoint_path if os.path.exists(best_checkpoint_path) else last_checkpoint_path
checkpoint = torch.load(checkpoint_path_to_load)
imp_classifier.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Loaded model from epoch {checkpoint['epoch']}")

imp_classifier.eval()
test_preds, test_labels, test_probs = [], [], []

with torch.no_grad():
    for batch in tqdm(imp_test_loader, desc="Testing"):
        pixel_values = batch['pixel_values'].to(imp_cfg.DEVICE)
        labels = batch['labels'].to(imp_cfg.DEVICE)

        # Forward pass
        logits = imp_classifier(pixel_values)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)

        # Collect results
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())  # Probability of abnormal class

# Calculate all metrics
test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='binary')
test_prec = precision_score(test_labels, test_preds, average='binary')
test_rec = recall_score(test_labels, test_preds, average='binary')
test_auc = roc_auc_score(test_labels, test_probs)

# Print results
print(f"\n{'='*60}")
print("STANDARD TEST RESULTS")
print(f"{'='*60}")
print(f"Accuracy:  {test_acc*100:.2f}%")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print(f"AUC-ROC:   {test_auc:.4f}")
print(f"\nConfusion Matrix:")
print(confusion_matrix(test_labels, test_preds))
print(f"\nClassification Report:")
print(classification_report(test_labels, test_preds, target_names=['Normal', 'Abnormal']))
print("="*60)

# ===========================================================================
# CELL 9: TEST-TIME AUGMENTATION (TTA) EVALUATION
# ===========================================================================
print(f"\n{'='*80}")
print("TEST EVALUATION WITH TEST-TIME AUGMENTATION (TTA)")
print(f"{'='*80}")

def predict_with_tta(model, image_path, processor, device, n_augmentations=8):
    """
    Perform Test-Time Augmentation for more robust predictions.

    Strategy:
    1. Make predictions on original image
    2. Make predictions on multiple augmented versions
    3. Average all predictions

    This typically improves accuracy by 1-3% by reducing variance.

    Args:
        model: Trained classifier
        image_path: Path to image
        processor: PaliGemma processor
        device: torch device
        n_augmentations: Number of augmented versions to use

    Returns:
        avg_pred: Averaged prediction probabilities [1, num_classes]
    """
    model.eval()

    # Load image
    try:
        image = Image.open(image_path).convert('RGB')
        if image.size != (224, 224):
            image = image.resize((224, 224), Image.LANCZOS)
    except:
        # Return uniform probabilities if image fails to load
        return torch.tensor([[0.5, 0.5]]).to(device)

    predictions = []

    # 1. Original image prediction
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        logits = model(inputs['pixel_values'].to(device))
        predictions.append(torch.softmax(logits, dim=1))

    # 2. Define augmentation transforms
    # These are deterministic transforms (not random) for consistency
    aug_transforms = [
        # Flips
        transforms.Compose([transforms.RandomHorizontalFlip(p=1.0)]),
        transforms.Compose([transforms.RandomVerticalFlip(p=1.0)]),
        transforms.Compose([
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.RandomVerticalFlip(p=1.0)
        ]),

        # Rotations
        transforms.Compose([
            transforms.Lambda(lambda img: transforms.functional.rotate(img, 10))
        ]),
        transforms.Compose([
            transforms.Lambda(lambda img: transforms.functional.rotate(img, -10))
        ]),

        # Color adjustments
        transforms.Compose([
            transforms.Lambda(lambda img: transforms.functional.adjust_brightness(img, 1.2))
        ]),
        transforms.Compose([
            transforms.Lambda(lambda img: transforms.functional.adjust_contrast(img, 1.2))
        ]),
    ]

    # 3. Apply augmentations and get predictions
    for i, transform in enumerate(aug_transforms[:n_augmentations-1]):
        try:
            aug_image = transform(image)
            inputs = processor(images=aug_image, return_tensors="pt")
            with torch.no_grad():
                logits = model(inputs['pixel_values'].to(device))
                predictions.append(torch.softmax(logits, dim=1))
        except:
            # Skip this augmentation if it fails
            continue

    # 4. Average all predictions
    if len(predictions) > 0:
        avg_pred = torch.stack(predictions).mean(dim=0)
    else:
        # Fallback if all augmentations failed
        avg_pred = torch.tensor([[0.5, 0.5]]).to(device)

    return avg_pred

# Run TTA evaluation
print("Running TTA predictions (this will take longer)...")
tta_preds, tta_labels, tta_probs = [], [], []

for idx in tqdm(range(len(test_df)), desc="TTA Testing"):
    row = test_df.iloc[idx]

    # Get TTA prediction
    probs = predict_with_tta(
        imp_classifier,
        row['image_path'],
        processor,
        imp_cfg.DEVICE,
        n_augmentations=8
    )

    # Extract prediction and probability
    pred = torch.argmax(probs, dim=1).item()
    prob_abnormal = probs[0, 1].item()

    tta_preds.append(pred)
    tta_labels.append(row['label'])
    tta_probs.append(prob_abnormal)

# Calculate TTA metrics
tta_acc = accuracy_score(tta_labels, tta_preds)
tta_f1 = f1_score(tta_labels, tta_preds, average='binary')
tta_prec = precision_score(tta_labels, tta_preds, average='binary')
tta_rec = recall_score(tta_labels, tta_preds, average='binary')
tta_auc = roc_auc_score(tta_labels, tta_probs)

# Print TTA results
print(f"\n{'='*60}")
print("TTA TEST RESULTS")
print(f"{'='*60}")
print(f"Accuracy:  {tta_acc*100:.2f}%")
print(f"Precision: {tta_prec:.4f}")
print(f"Recall:    {tta_rec:.4f}")
print(f"F1 Score:  {tta_f1:.4f}")
print(f"AUC-ROC:   {tta_auc:.4f}")
print(f"\nConfusion Matrix:")
print(confusion_matrix(tta_labels, tta_preds))
print(f"\nClassification Report:")
print(classification_report(tta_labels, tta_preds, target_names=['Normal', 'Abnormal']))
print("="*60)

# ===========================================================================
# CELL 10: COMPARISON AND RESULTS SUMMARY
# ===========================================================================
print(f"\n{'='*80}")
print("RESULTS COMPARISON")
print(f"{'='*80}")

# Create comparison table
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC'],
    'Original Model': [
        f"{0.8065*100:.2f}%",
        f"{0.7632:.4f}",
        f"{0.9062:.4f}",
        f"{0.8286:.4f}",
        f"{0.8479:.4f}"
    ],
    'Improved (Standard)': [
        f"{test_acc*100:.2f}%",
        f"{test_prec:.4f}",
        f"{test_rec:.4f}",
        f"{test_f1:.4f}",
        f"{test_auc:.4f}"
    ],
    'Improved (TTA)': [
        f"{tta_acc*100:.2f}%",
        f"{tta_prec:.4f}",
        f"{tta_rec:.4f}",
        f"{tta_f1:.4f}",
        f"{tta_auc:.4f}"
    ],
    'Improvement': [
        f"{(tta_acc - 0.8065)*100:.2f}%",
        f"{(tta_prec - 0.7632):.4f}",
        f"{(tta_rec - 0.9062):.4f}",
        f"{(tta_f1 - 0.8286):.4f}",
        f"{(tta_auc - 0.8479):.4f}"
    ]
})

print(comparison_df.to_string(index=False))

# Save all results
results_summary = {
    'original_model': {
        'test_accuracy': 0.8065,
        'test_precision': 0.7632,
        'test_recall': 0.9062,
        'test_f1': 0.8286,
        'test_auc': 0.8479
    },
    'improved_standard': {
        'test_accuracy': float(test_acc),
        'test_precision': float(test_prec),
        'test_recall': float(test_rec),
        'test_f1': float(test_f1),
        'test_auc': float(test_auc)
    },
    'improved_tta': {
        'test_accuracy': float(tta_acc),
        'test_precision': float(tta_prec),
        'test_recall': float(tta_rec),
        'test_f1': float(tta_f1),
        'test_auc': float(tta_auc)
    },
    'training_history': {
        'train': train_history,
        'val': val_history
    }
}

# Save to JSON
results_path = os.path.join(imp_cfg.METRICS_DIR, 'results_comparison.json')
with open(results_path, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"\n✅ Results saved to {results_path}")

# ===========================================================================
# CELL 11: SAMPLE PREDICTIONS WITH COMPARISON
# ===========================================================================
print(f"\n{'='*80}")
print("SAMPLE PREDICTIONS COMPARISON")
print(f"{'='*80}")

# Select random test samples
np.random.seed(42)
sample_indices = np.random.choice(len(test_df), min(10, len(test_df)), replace=False)

print(f"\n{'Sample':<8} {'True Label':<12} {'Standard Pred':<15} {'TTA Pred':<15} {'Improvement':<12}")
print("-" * 80)

for i, idx in enumerate(sample_indices):
    row = test_df.iloc[idx]
    true_label = "Normal" if row['label'] == 0 else "Abnormal"

    # Get standard prediction
    standard_pred = "Normal" if test_preds[idx] == 0 else "Abnormal"
    standard_conf = test_probs[idx] if test_preds[idx] == 1 else (1 - test_probs[idx])

    # Get TTA prediction
    tta_pred = "Normal" if tta_preds[idx] == 0 else "Abnormal"
    tta_conf = tta_probs[idx] if tta_preds[idx] == 1 else (1 - tta_probs[idx])

    # Check if predictions match true label
    standard_correct = "✅" if standard_pred == true_label else "❌"
    tta_correct = "✅" if tta_pred == true_label else "❌"

    # Determine improvement
    if standard_correct == "❌" and tta_correct == "✅":
        improvement = "TTA Fixed!"
    elif standard_correct == "✅" and tta_correct == "✅":
        improvement = "Both Good"
    elif standard_correct == "✅" and tta_correct == "❌":
        improvement = "TTA Worse"
    else:
        improvement = "Both Wrong"

    print(f"{i+1:<8} {true_label:<12} "
          f"{standard_pred} {standard_correct} ({standard_conf*100:.1f}%)"
          f"{'':<3} {tta_pred} {tta_correct} ({tta_conf*100:.1f}%)"
          f"{'':<3} {improvement:<12}")

print("\n" + "="*80)
print("ALL EXPERIMENTS COMPLETE!")
print("="*80)
print(f"\n📊 Final Performance Summary:")
print(f"   Original Model:        Acc={0.8065*100:.2f}%, F1={0.8286:.4f}, AUC={0.8479:.4f}")
print(f"   Improved (Standard):   Acc={test_acc*100:.2f}%, F1={test_f1:.4f}, AUC={test_auc:.4f}")
print(f"   Improved (TTA):        Acc={tta_acc*100:.2f}%, F1={tta_f1:.4f}, AUC={tta_auc:.4f}")
print(f"\n💾 All results saved to: {imp_cfg.OUTPUT_DIR}")
print("="*80)

✅ Enhanced configuration created
✅ Improved dataset class created
✅ Improved classifier architecture defined

CREATING IMPROVED DATASETS
Creating datasets with augmentation...
✅ Train dataset: 219 samples (with augmentation)
✅ Val dataset: 25 samples (no augmentation)
✅ Test dataset: 62 samples (no augmentation)
✅ Train loader: 55 batches
✅ Val loader: 7 batches
✅ Test loader: 16 batches

CREATING IMPROVED MODEL
Building improved classifier...
Detecting vision model architecture...
  Vision output shape: torch.Size([1, 256, 1152])
  Vision output dtype: torch.float32
  Sequence length: 256
  Feature dimension: 1152

📊 Model Statistics:
  Trainable parameters: 1,447,042 (0.35%)
  Total parameters: 413,889,394
  GPU Memory: 13.44 GB
✅ Improved classifier ready

SETTING UP IMPROVED TRAINING
✅ Optimizer: AdamW
   Initial LR: 3.00e-04
   Max LR: 5.00e-04
   Weight decay: 0.01
✅ Scheduler: OneCycleLR
   Warmup: 4 epochs
   Steps per epoch: 55

STARTING IMPROVED TRAINING

Epoch 1/15


Training:   0%|          | 0/55 [00:00<?, ?it/s]

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y

Train - Loss: 0.7391 | Acc: 54.79% | F1: 0.6689 | Prec: 0.5319 | Rec: 0.9009


Validation:   0%|          | 0/7 [00:00<?, ?it/s]

...

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y


STANDARD TEST RESULTS
Accuracy:  72.58%
Precision: 0.8000
Recall:    0.6250
F1 Score:  0.7018
AUC-ROC:   0.8385

Confusion Matrix:
[[25  5]
 [12 20]]

Classification Report:
              precision    recall  f1-score   support

      Normal       0.68      0.83      0.75        30
    Abnormal       0.80      0.62      0.70        32

    accuracy                           0.73        62
   macro avg       0.74      0.73      0.72        62
weighted avg       0.74      0.73      0.72        62


TEST EVALUATION WITH TEST-TIME AUGMENTATION (TTA)
Running TTA predictions (this will take longer)...


TTA Testing:   0%|          | 0/62 [00:00<?, ?it/s]

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.
Y


TTA TEST RESULTS
Accuracy:  74.19%
Precision: 0.7857
Recall:    0.6875
F1 Score:  0.7333
AUC-ROC:   0.8375

Confusion Matrix:
[[24  6]
 [10 22]]

Classification Report:
              precision    recall  f1-score   support

      Normal       0.71      0.80      0.75        30
    Abnormal       0.79      0.69      0.73        32

    accuracy                           0.74        62
   macro avg       0.75      0.74      0.74        62
weighted avg       0.75      0.74      0.74        62


RESULTS COMPARISON
   Metric Original Model Improved (Standard) Improved (TTA) Improvement
 Accuracy         80.65%              72.58%         74.19%      -6.46%
Precision         0.7632              0.8000         0.7857      0.0225
   Recall         0.9062              0.6250         0.6875     -0.2187
 F1 Score         0.8286              0.7018         0.7333     -0.0953
  AUC-ROC         0.8479              0.8385         0.8375     -0.0104

✅ Results saved to /content/drive/MyDrive/VIA_Pali